# Model Problem on the Unit Disk

Let $\Omega$ be the **unit disk** in $\mathbb{R}^2$.

We want to solve the model problem

$$
- \Delta u = f \quad \text{in } \Omega, \qquad u = u_D \quad \text{on } \partial \Omega,
$$

where $\Omega$ is embedded in a **background domain** $(-1.25,\ 1.25)^2$.

We consider the Dirichlet problem with prescribed boundary data $u_D$ and a right-hand side function $f$.



In [11]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

In [12]:
maxh = 0.1
# Finite element space order
order = 1
# Stabilization parameter for ghost-penalty
gamma_stab = 0.1
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

In [13]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [14]:
r = sqrt(x**2 + y**2)
levelset = r-1

exact = sin(x+y)
coeff_f = - (exact.Diff(x).Diff(x) + exact.Diff(y).Diff(y)).Compile()

In [20]:
lis = [(0.0,10*order**2), (1.0,10*order**2), (1.0, order**2),(100.0, 10*order**2)]
H = [0.2,0.1, 0.05, 0.025, 0.0125]
l2error = []
for maxh in H:
        maxh = maxh
        square = SplineGeometry()
        square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
        ngmesh = square.GenerateMesh(maxh=maxh)
        mesh = Mesh(ngmesh)

        lsetp1 = GridFunction(H1(mesh))
        InterpolateToP1(levelset,lsetp1)
            # Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
        ci = CutInfo(mesh, lsetp1)
        hasneg = ci.GetElementsOfType(HASNEG)
        neg = ci.GetElementsOfType(NEG)
        hasif = ci.GetElementsOfType(IF)
        haspos = ci.GetElementsOfType(HASPOS)
        ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
        patch_elements = ci.GetElementsOfType(ANY)


        #band_elements = hasneg | hasif
        #ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
        Vh = H1(mesh, order=order, dirichlet=[], dgjumps=True)
        gfu = GridFunction(Vh)
        u, v = Vh.TrialFunction(), Vh.TestFunction()
        h = specialcf.mesh_size
        n = Normalize(grad(lsetp1))

            # integration domains:
        dx = dCut(lsetp1, NEG, definedonelements=hasneg)
        ds = dCut(lsetp1, IF, definedonelements=hasif)
        dw = dFacetPatch(definedonelements=ba_facets)
        for i in lis:
                lambda_nitsche = i[0]
                gamma_stab = i[1]

                a = BilinearForm(Vh, symmetric=True)
                # Diffusion term
                a += grad(u) * grad(v) * dx
                # Nitsche term
                a += -grad(u) * n * v * ds
                a += -grad(v) * n * u * ds
                a += (lambda_nitsche / h) * u * v * ds
                # Ghost penalty stabilization (near the boundary)
                a += gamma_stab / h**2 * (u - u.Other()) * (v - v.Other()) * dw

                # R.h.s. term:
                f = LinearForm(Vh)
                f += coeff_f * v * dx
                f += exact * (-grad(v) * n + (lambda_nitsche / h) * v) * ds

                # Assemble system
                a.Assemble()
                f.Assemble()

                # Solve linear system
                gfu.vec.data = a.mat.Inverse(Vh.FreeDofs(),inverse="sparsecholesky") * f.vec
                l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
                #print("L2 Error: {0}".format(l2error), "meshzise: {0}".format(maxh), "lambda_nitsche: {0}".format(lambda_nitsche), "gamma_stab: {0}".format(gamma_stab))

In [21]:
rows = [l2error[i:i+4] for i in range(0, len(l2error), 4)]

# Ausgabe als Tabelle
for row in rows:
    print("{:<10} {:<15} {:<10} {:<10}".format(*row))

0.08119239183651081 0.05809140246549389 0.013209676504594904 0.01616055942337263
0.008551514804200154 0.030650677369484566 0.005916475646148908 0.0051087932223570975
0.012084489155784613 0.004727411141470108 0.0006268600344721731 0.0014563609118965265
0.00044981144533954296 0.0010948853148318588 0.00016014123326247922 0.0003926627148825263
0.00011199539534279778 0.00015364487930839748 0.00012085040217367153 0.00010301019031779888


In [ ]:
# visualization:
DrawDC(lsetp1, gfu, 0, mesh, "uh")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
lis = [(0.0,10*order**2), (1.0,10*order**2), (1.0, order**2),(100.0, 10*order**2)]
H = [0.2,0.1, 0.05, 0.025, 0.0125, 0.00625]
l2error = []
for maxh in H:
        maxh = maxh
        square = SplineGeometry()
        square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
        ngmesh = square.GenerateMesh(maxh=maxh)
        mesh = Mesh(ngmesh)

        lsetp1 = GridFunction(H1(mesh))
        InterpolateToP1(levelset,lsetp1)
            # Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
        ci = CutInfo(mesh, lsetp1)
        hasneg = ci.GetElementsOfType(HASNEG)
        hasif = ci.GetElementsOfType(IF)
        ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
        Vhbase = H1(mesh, order=order, dirichlet=[], dgjumps=True)
        Vh = Restrict(Vhbase, hasneg)
        gfu = GridFunction(Vh)
        u, v = Vh.TrialFunction(), Vh.TestFunction()
        h = specialcf.mesh_size
        n = Normalize(grad(lsetp1))

            # integration domains:
        dx = dCut(lsetp1, NEG, definedonelements=hasneg)
        ds = dCut(lsetp1, IF, definedonelements=hasif)
        dw = dFacetPatch(definedonelements=ba_facets)
        for i in lis:
                lambda_nitsche = i[0]
                gamma_stab = i[1]

                a = BilinearForm(Vh, symmetric=True)
                # Diffusion term
                a += grad(u) * grad(v) * dx
                # Nitsche term
                a += -grad(u) * n * v * ds
                a += -grad(v) * n * u * ds
                a += (lambda_nitsche / h) * u * v * ds
                # Ghost penalty stabilization (near the boundary)
                a += gamma_stab / h**2 * (u - u.Other()) * (v - v.Other()) * dw

                # R.h.s. term:
                f = LinearForm(Vh)
                f += coeff_f * v * dx
                f += exact * (-grad(v) * n + (lambda_nitsche / h) * v) * ds

                # Assemble system
                a.Assemble()
                f.Assemble()

                # Solve linear system
                gfu.vec.data = a.mat.Inverse(Vh.FreeDofs(),inverse="sparsecholesky") * f.vec
                l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
                #print("L2 Error: {0}".format(l2error), "meshzise: {0}".format(maxh), "lambda_nitsche: {0}".format(lambda_nitsche), "gamma_stab: {0}".format(gamma_stab))

In [ ]:
rows = [l2error[i:i+4] for i in range(0, len(l2error), 4)]

# Ausgabe als Tabelle
for row in rows:
    print("{:<10} {:<15} {:<10} {:<10}".format(*row))

0.04768965191994027 0.1595572023060386 0.015432806959200102 0.015139445869359242
0.011680241449170663 0.04348443347571346 0.010853841344924195 0.0032566188930135084
0.0011797325860983146 0.019066069605837507 0.0010879912235682973 0.0006297603550723777
0.00031147999015272444 0.0018257370510630167 0.0001729711297059448 0.00012454120423134314
7.875946113218658e-05 0.001049452189266725 0.0004808407805670847 2.629443270288592e-05
1.0915879855848704e-05 0.0001153064676045657 5.849816994253416e-06 5.922306982604988e-06


Yes the results are indicating the convergencerate we had in our analysis